In [137]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [138]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
submission = pd.read_csv('SampleSubmission.csv')
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,1/6/2013,M,U,1991,748L,QZYX,90QI,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,1/8/2019,M,M,1990,1X1H,BP09,56SI,0,0,...,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,1/8/2019,M,M,1990,748L,NO3L,T4MS,0,0,...,0,0,0,0,0,0,1,1,0,0


In [139]:
print(train.shape)
print(test.shape)
print(submission.shape)

(29132, 29)
(10000, 29)
(210000, 2)


In [140]:
#train = train.fillna(0,inplace=True)
#test = test.fillna(0,inplace=True)

In [141]:
test

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,F86J5PC,1/12/2018,M,M,1984,94KC,DZRV,90QI,0,0,...,0,0,0,0,0,0,0,0,0,0
1,H6141K3,1/10/2019,M,M,1996,1X1H,J9SY,90QI,0,0,...,0,0,0,1,0,0,0,0,0,0
2,RBAYUXZ,1/1/2020,F,W,1968,UAOD,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
3,KCBILBQ,1/2/2019,M,M,1989,94KC,2A7I,T4MS,0,0,...,0,0,0,0,0,0,0,0,0,0
4,LSEC1ZJ,1/2/2020,F,M,1982,UAOD,0KID,T4MS,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0GMU5UH,1/4/2019,F,M,1986,748L,2A7I,T4MS,0,0,...,0,0,0,0,1,0,1,0,0,0
9996,I9W11CD,1/8/2019,M,M,1980,UAOD,0KID,T4MS,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,42WTEGT,1/4/2018,M,M,1993,748L,SST3,56SI,0,0,...,0,0,0,1,0,0,0,0,0,0
9998,8EKC4O9,1/5/2015,F,U,1981,49BM,2A7I,T4MS,0,0,...,0,0,0,0,0,0,0,0,0,0


## Preprocessing

In [142]:
train.columns

Index(['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3'],
      dtype='object')

In [143]:
product_train = train[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]

In [144]:
#Axis Transformation
train = train.melt(id_vars=train.columns[:8], value_vars=product_train, var_name = "PCODE", value_name="Label" )
test = test.melt(id_vars=test.columns[:8], value_vars=product_train, var_name = "PCODE", value_name="Label" )

#A merger
train['combiner']='x'

#Set
data=pd.concat([train,test],sort=False).reset_index(drop=True)

train=data[data.combiner.notnull()].reset_index(drop=True);test=data[data.combiner.isna()].reset_index(drop=True)
train.drop('combiner', inplace=True, axis=1)
test.drop(['Label','combiner'], inplace=True, axis=1)

#Concatenate new column towards submission requirement
train['ID X PCODE'] = train['ID'] + ' X ' + train['PCODE']
test['ID X PCODE'] = test['ID'] + ' X ' + test['PCODE']

In [145]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,Label,ID X PCODE
0,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,P5DA,0,4WKQSBB X P5DA
1,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,P5DA,0,CP5S02H X P5DA
2,2YKDILJ,1/6/2013,M,U,1991,748L,QZYX,90QI,P5DA,0,2YKDILJ X P5DA
3,2S9E81J,1/8/2019,M,M,1990,1X1H,BP09,56SI,P5DA,0,2S9E81J X P5DA
4,BHDYVFT,1/8/2019,M,M,1990,748L,NO3L,T4MS,P5DA,0,BHDYVFT X P5DA


In [146]:
test.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
0,F86J5PC,1/12/2018,M,M,1984,94KC,DZRV,90QI,P5DA,F86J5PC X P5DA
1,H6141K3,1/10/2019,M,M,1996,1X1H,J9SY,90QI,P5DA,H6141K3 X P5DA
2,RBAYUXZ,1/1/2020,F,W,1968,UAOD,2A7I,T4MS,P5DA,RBAYUXZ X P5DA
3,KCBILBQ,1/2/2019,M,M,1989,94KC,2A7I,T4MS,P5DA,KCBILBQ X P5DA
4,LSEC1ZJ,1/2/2020,F,M,1982,UAOD,0KID,T4MS,P5DA,LSEC1ZJ X P5DA


#### Dealing with Categorical columns

In [147]:
train.index = pd.to_datetime(train.index)

In [148]:
train['Day'] = train['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
train['Month'] = train['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
train['Year'] = train['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
train.drop('join_date', axis=1, inplace=True)

test['Day'] = test['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
test['Month'] = test['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
test['Year'] = test['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
test.drop('join_date', axis=1, inplace=True)

#train['date_diff'] = train['date3'] - train['birth_year']
#test['date_diff'] = test['date3'] - test['birth_year']

In [149]:
train.head()

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,Label,ID X PCODE,Day,Month,Year
1970-01-01 00:00:00.000000000,4WKQSBB,F,M,1987,1X1H,2A7I,T4MS,P5DA,0,4WKQSBB X P5DA,1.0,2.0,2019.0
1970-01-01 00:00:00.000000001,CP5S02H,F,M,1981,UAOD,2A7I,T4MS,P5DA,0,CP5S02H X P5DA,1.0,6.0,2019.0
1970-01-01 00:00:00.000000002,2YKDILJ,M,U,1991,748L,QZYX,90QI,P5DA,0,2YKDILJ X P5DA,1.0,6.0,2013.0
1970-01-01 00:00:00.000000003,2S9E81J,M,M,1990,1X1H,BP09,56SI,P5DA,0,2S9E81J X P5DA,1.0,8.0,2019.0
1970-01-01 00:00:00.000000004,BHDYVFT,M,M,1990,748L,NO3L,T4MS,P5DA,0,BHDYVFT X P5DA,1.0,8.0,2019.0


In [150]:
train.Year.value_counts()

2019.0    186984
2018.0    154392
2017.0     92715
2020.0     66318
2016.0     44520
2015.0     19110
2014.0     15477
2013.0     12831
2012.0     11235
2011.0      6615
2010.0      1533
Name: Year, dtype: int64

In [151]:
train.columns

Index(['ID', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE', 'Label',
       'ID X PCODE', 'Day', 'Month', 'Year'],
      dtype='object')

In [152]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()

# Encode the train categorical columns

train['sex'] = enc.fit_transform(train['sex'])
train['marital_status'] = enc.fit_transform(train['marital_status'])
train['branch_code'] = enc.fit_transform(train['branch_code'])
train['occupation_code'] = enc.fit_transform(train['occupation_code'])
train['occupation_category_code'] = enc.fit_transform(train['occupation_category_code'])
train['PCODE'] = enc.fit_transform(train['PCODE'])


#Doing the same thing for test
test['sex'] = enc.fit_transform(test['sex'])
test['marital_status'] = enc.fit_transform(test['marital_status'])
test['branch_code'] = enc.fit_transform(test['branch_code'])
test['occupation_code'] = enc.fit_transform(test['occupation_code'])
test['occupation_category_code'] = enc.fit_transform(test['occupation_category_code'])
test['PCODE'] = enc.fit_transform(test['PCODE'])


In [161]:
train.Day = train.Day.dropna(axis=0)
train.isnull().sum()

ID                           0
sex                          0
marital_status               0
birth_year                   0
branch_code                  0
occupation_code              0
occupation_category_code     0
PCODE                        0
Label                        0
ID X PCODE                   0
Day                         42
Month                       42
Year                        42
dtype: int64

In [154]:
feature= train.drop(['ID','ID X PCODE','Label'],1)
target = train.Label


In [155]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

Xtrain, Xtest,ytrain,ytest = train_test_split(feature,target, test_size=0.3)
scaler = MinMaxScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.fit_transform(Xtest)

clf = LogisticRegression()
clf.fit(Xtrain,ytrain)
score = clf.score(Xtrain,ytrain)




ValueError: Input contains NaN, infinity or a value too large for dtype('float64').